In [ ]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import tensorflow as tf
import _pickle as pkl

from utils import *
from FCNN import FCNN
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression

## Read Datasets

LFW+

In [ ]:
dataset_address = './LFWA+/lfw'
dataset = get_dataset(dataset_address)
lfw_raw, label_list = get_image_paths_and_labels(dataset)
with open("./dataset_description.pkl","rb") as foo:
    dc = pkl.load(foo)
lfw_list = []
for im in dc['image_list']:
    lfw_list.append(os.path.join(dataset_address, im))
lfw_attributes = (dc['attributes']>0).astype(int)
lfw_labels = (lfw_attributes[:,0]>0).astype(int)
lfw_latent_vars = dc['latent_vars'] ##Latent representation of the image in the VAE trained on CelebA
lfw_data = ((read_images(lfw_list) - 127.5)/128, lfw_labels, lfw_latent_vars)

In [ ]:
sex = lfw_attributes[:,0]
skin = lfw_attributes[:,3]
dictionary = dc
permuted_idxs = np.random.permutation(np.arange(len(lfw_data[0])))
idxs_val, idxs_test = permuted_idxs[:6263], permuted_idxs[6263:]
x_val, y_val, a_val = lfw_data[0][idxs_val], lfw_data[1][idxs_val], lfw_attributes[:,[3,38,40]][idxs_val]
x_test, y_test, a_test = lfw_data[0][idxs_test], lfw_data[1][idxs_test], lfw_attributes[:,[3,38,40]][idxs_test]

## Load trained network

In [ ]:
tf.reset_default_graph()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1.0)
gender_ckpt = "./model-20180428-135113_male.ckpt-111"
net = FCNN()
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=False))
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
net.load_model(sess, gender_ckpt, initialize = False)

## Network & VAE Representation

In [ ]:
prelogits_val = np.zeros((len(x_val),128))
batch_size = 24
for i in range(int(np.ceil(len(x_val)/batch_size))):
    prelogits_val[i*batch_size:i*batch_size+batch_size] = sess.run(
        net.prelogits ,feed_dict={net.in_ph:x_val[i*batch_size:i*batch_size+batch_size]})
lts_val = dictionary['latent_vars'][idxs_val] #VAE representation
prelogits_test = np.zeros((len(x_test),128))
for i in range(int(np.ceil(len(x_test)/batch_size))):
    prelogits_test[i*batch_size:i*batch_size+batch_size] = sess.run(
        net.prelogits, feed_dict={net.in_ph:x_test[i*batch_size:i*batch_size+batch_size]})
lts_test = dictionary['latent_vars'][idxs_test] #VAE representation

## Black-box audting 

In [ ]:
def sess_run(result, x, l, sess):
    num = x.shape[0]
    num_batch = np.ceil(num/200).astype(int)
    output = np.zeros(num)
    for batch in range(num_batch):
        output[batch*200:(batch+1)*200] = sess.run(result, feed_dict={net.in_ph:x[batch*200:(batch+1)*200],
                                                                         latent_ph:l[batch*200:(batch+1)*200]})       
    return output

In [ ]:
control = tf.cast(tf.greater(net.output[:,1],net.output[:,0]), tf.float32)
noharm = [control, 1 - control, control + 1 - control]
latent_val, latent_test = lts_val, lts_test
dim = latent_val.shape[-1]
latent_ph = tf.placeholder(tf.float32, shape=(None, dim), name="latent_var")
logits = net.output[:,1] - net.output[:,0]
max_T = 100
thresh = 1e-4 # Hyper-parameter
temp_harm = []
for s in noharm:
    temp_harm.append(sess_run(s, x_val, latent_val, sess))

In [ ]:
def res(p, y):
    return y * ((p>=0.1)/(p + 1e-20) + (p<0.1) * (20 - 100  * p)) +\
(1-y) * ((p < 0.9)/(1 - p + 1e-20) + (p>=0.9) * (100 * p - 80))
plt.plot(np.arange(0, 1.0, 0.001), res(np.arange(0., 1, 0.001), 0))
plt.plot(np.arange(0, 1.0, 0.001), res(np.arange(0., 1, 0.001), 1))
plt.legend(['0', '1'])
plt.xlabel('P')
plt.ylabel('Residual')

In [ ]:
###### PREVIOUS CODE ######

best_epoch, best_acc = -1,0
(idxs1, idxs2, _), _ = split_data(np.arange(len(idxs_val)), ratio=[0.7,0.3,0.])
coeffs = []
for t in range(max_T):
    control = tf.cast(tf.greater(net.output[:,1], net.output[:,0]), tf.float32)
    noharm = [control, 1 - control, control + 1 - control]
    probs_heldout = sess_run(tf.nn.sigmoid(logits), x_val[idxs2], latent_val[idxs2], sess)
    heldout_loss = np.mean(-y_val[idxs2] * np.log(probs_heldout + 1e-20) - (1-y_val[idxs2]) * np.log(1-probs_heldout + 1e-20))
    heldout_acc =  np.mean((probs_heldout>0.5)==y_val[idxs2])
    probs = sess_run(tf.nn.sigmoid(logits), x_val, latent_val ,sess)
    val_loss = np.mean(-y_val * np.log(probs + 1e-20) - (1 - y_val) * np.log(1 - probs + 1e-20))
    val_acc = np.mean((probs > 0.5) == y_val)
    if heldout_acc > best_acc:
        best_epoch = t
        best_acc = heldout_acc
        best_logits = logits
    delta = res(probs,y_val)
    residual = probs - y_val
    for i, s in enumerate(noharm):
        temp_s = sess_run(noharm[i], x_val[idxs1], latent_val[idxs1], sess)
        temp_s_heldout = sess_run(noharm[i], x_val[idxs2], latent_val[idxs2], sess)
        samples1 = np.where(temp_s == 1)[0]
        samples2 = np.where(temp_s_heldout == 1)[0]
        clf = Ridge(alpha=1)
        clf.fit(latent_val[idxs1][samples1],delta[idxs1][samples1])
        clf_prediction = clf.predict(latent_val[idxs2][samples2])
        corr = np.mean(clf_prediction * residual[idxs2][samples2])
        print(t, i, corr)
        if corr > 1e-4:
            coeffs.append(clf.coef_)
            h = (tf.matmul(latent_ph, tf.constant(np.expand_dims(clf.coef_,-1),
                                                  dtype=tf.float32))[:,0] + clf.intercept_)
            logits -= .1 * h * s
            break
    if i==2:
        break

In [ ]:
###### DP VERSION ######


# PRIVACY ACCOUNTING - Added for DP Multiaccuracy

# Set privacy budget (ε, δ)
total_epsilon = 1.0  # Total epsilon budget
total_delta = 1e-5   # Total delta budget

# Initialize privacy accountant to track budget consumption
privacy_accountant = PrivacyAccountant(total_epsilon, total_delta)

# Auditor hyperparameters (these affect privacy)
auditor_noise_multiplier = 1.0  # Controls privacy strength (higher = more private)
auditor_num_steps = 200
auditor_batch_size = 128
auditor_clipping_norm = 1.0

best_epoch, best_acc = -1,0
(idxs1, idxs2, _), _ = split_data(np.arange(len(idxs_val)), ratio=[0.7,0.3,0.])
coeffs = []
for t in range(max_T):
    control = tf.cast(tf.greater(net.output[:,1], net.output[:,0]), tf.float32)
    noharm = [control, 1 - control, control + 1 - control]
    probs_heldout = sess_run(tf.nn.sigmoid(logits), x_val[idxs2], latent_val[idxs2], sess)
    heldout_loss = np.mean(-y_val[idxs2] * np.log(probs_heldout + 1e-20) - (1-y_val[idxs2]) * np.log(1-probs_heldout + 1e-20))
    heldout_acc =  np.mean((probs_heldout>0.5)==y_val[idxs2])
    probs = sess_run(tf.nn.sigmoid(logits), x_val, latent_val ,sess)
    val_loss = np.mean(-y_val * np.log(probs + 1e-20) - (1 - y_val) * np.log(1 - probs + 1e-20))
    val_acc = np.mean((probs > 0.5) == y_val)
    if heldout_acc > best_acc:
        best_epoch = t
        best_acc = heldout_acc
        best_logits = logits
    delta = res(probs, y_val)
    residual = probs - y_val
    for i, s in enumerate(noharm):
        temp_s = sess_run(noharm[i], x_val[idxs1], latent_val[idxs1], sess)
        temp_s_heldout = sess_run(noharm[i], x_val[idxs2], latent_val[idxs2], sess)
        samples1 = np.where(temp_s == 1)[0]
        samples2 = np.where(temp_s_heldout == 1)[0]

        # Features / targets used to train the auditor
        X_train = latent_val[idxs1][samples1]          # (n1, d)
        y_train = delta[idxs1][samples1]               # (n1,)

        # PRIVACY BUDGET COMPUTATION - Split between auditor and correlation query
    
        dataset_size = len(X_train)
        delta_per_round = total_delta / max_T  # Allocate delta evenly across max rounds
        
        # Compute privacy cost for auditor training (DP-SGD)
        epsilon_audit_t = compute_epsilon_dp_sgd(
            auditor_noise_multiplier, 
            dataset_size, 
            auditor_num_steps, 
            auditor_batch_size, 
            delta_per_round / 2.0  
        )
        delta_audit_t = delta_per_round / 2.0
        
        # Compute privacy cost for correlation query
        # We need to set epsilon_corr_t = epsilon_audit_t to satisfy Proposition 1
        # The actual noise in compute_noisy_correlation will be calibrated based on epsilon_corr_t
        epsilon_corr_t = epsilon_audit_t  # Equal split 
        delta_corr_t = delta_per_round / 2.0
        
        
        epsilon_per_round = epsilon_audit_t + epsilon_corr_t
        
        # Check if we have enough privacy budget remaining
        if not privacy_accountant.can_allocate(epsilon_per_round, delta_per_round):
            print(f"Privacy budget exhausted at round {t}. Consumed: "
                  f"ε={privacy_accountant.consumed_epsilon:.4f}, "
                  f"δ={privacy_accountant.consumed_delta:.2e}")
            print(f"Remaining: {privacy_accountant.get_remaining_budget()}")
            break  # Stop if budget exhausted
        

        # DP auditor (class implemented in utils.py)
        d = X_train.shape[1]
        auditor = DPLinearAuditor(
            feature_dim=d,
            learning_rate=0.1,
            num_steps=auditor_num_steps,
            batch_size=auditor_batch_size,
            clipping_norm=auditor_clipping_norm,
            noise_multiplier=auditor_noise_multiplier,   # controls privacy strength
            seed=42
        )
        auditor.fit(X_train, y_train)

        
        #PRIVACY BUDGET ALLOCATION - Split between auditor and correlation
        # Allocate privacy budget for this round with explicit split
        if not privacy_accountant.allocate_round(epsilon_audit_t, epsilon_corr_t, 
                                                  delta_audit_t, delta_corr_t):
            print(f"Failed to allocate privacy budget at round {t}")
            break
        
        # Print privacy consumption
        consumed_eps, consumed_delta = privacy_accountant.get_consumed_budget()
        remaining_eps, remaining_delta = privacy_accountant.get_remaining_budget()
        print(f"Round {t}, Subgroup {i}: Privacy consumed - "
              f"ε={consumed_eps:.4f}/{total_epsilon:.4f} "
              f"(audit: {epsilon_audit_t:.4f}, corr: {epsilon_corr_t:.4f}), "
              f"δ={consumed_delta:.2e}/{total_delta:.2e}, "
              f"Remaining: ε={remaining_eps:.4f}, δ={remaining_delta:.2e}")
        

        #NOISY CORRELATION QUERY 
        # Use the training set for the correlation query 
        # The correlation is computed on the same dataset used for training
        correlation_clipping_bound = 1.0  # B: clipping bound for correlation scores
        corr = compute_noisy_correlation(
            auditor=auditor,
            X=X_train,  # Use training set for correlation query
            residuals=residual[idxs1][samples1],  # Residuals on training set
            clipping_bound=correlation_clipping_bound,
            epsilon=epsilon_corr_t,  # Privacy budget for correlation query
            delta=delta_corr_t,      # Privacy budget for correlation query
            seed=42 + t * 10 + i     # Different seed per round/subgroup
        )
        print(t, i, f"DP correlation: {corr:.6f}")

        if corr > 1e-4:
            # DPLinearAuditor learns only a weight vector w (no bias term)
            w = auditor.w      # shape (d,)
            h = tf.matmul(
                latent_ph,
                tf.constant(w.reshape(-1, 1), dtype=tf.float32)
            )[:, 0]  # h(x) = w^T z

            logits -= 0.1 * h * s
            break
    if i == 2:
        break


# FINAL PRIVACY REPORT 

consumed_eps, consumed_delta = privacy_accountant.get_consumed_budget()
print(f"\n=== Privacy Budget Summary ===")
print(f"Total budget: ε={total_epsilon:.4f}, δ={total_delta:.2e}")
print(f"Consumed: ε={consumed_eps:.4f}, δ={consumed_delta:.2e}")
print(f"Remaining: ε={total_epsilon - consumed_eps:.4f}, "
      f"δ={total_delta - consumed_delta:.2e}")
print(f"Number of rounds: {len(privacy_accountant.rounds)}")



## Result

In [ ]:
probs = sess_run(net.output[:,1] - net.output[:,0], x_test, latent_test, sess)
groups = ['all', 'F', 'M', 'B', 'N', 'BF', 'BM', 'NF', 'NM']
errs = []
idxs = np.where((skin[idxs_test]>-1) * (sex[idxs_test]>-10))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]>-1) * (sex[idxs_test]==0))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]>-1) * (sex[idxs_test]==1))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==1) * (sex[idxs_test]>-10))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==0) * (sex[idxs_test]>-10))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==1) * (sex[idxs_test]==0))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==1) * (sex[idxs_test]==1))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==0) * (sex[idxs_test]==0))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==0) * (sex[idxs_test]==1))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
output = ''
for group, err in zip(groups, errs):
    output += group + ': ' + str(round(err, 1)) + ' & '
print('Original: ', output)

In [ ]:
probs = sess_run(tf.nn.sigmoid(best_logits), x_test, latent_test, sess)
groups = ['all', 'F', 'M', 'B', 'N', 'BF', 'BM', 'NF', 'NM']
errs = []
idxs = np.where((skin[idxs_test]>-1) * (sex[idxs_test]>-10))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]>-1) * (sex[idxs_test]==0))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]>-1) * (sex[idxs_test]==1))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==1) * (sex[idxs_test]>-10))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==0) * (sex[idxs_test]>-10))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==1) * (sex[idxs_test]==0))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==1) * (sex[idxs_test]==1))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==0) * (sex[idxs_test]==0))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
idxs = np.where((skin[idxs_test]==0) * (sex[idxs_test]==1))[0]
errs.append(100 * np.mean((probs[idxs]>0.5)!=y_test[idxs]))
output = ''
for group, err in zip(groups, errs):
    output += group + ': ' + str(round(err, 1)) + ' & '
print('MultiAccuracy Boost: ', output)